# S3から入力データや実験ソースコードを用意する
ここでは、S3のストレージにあるデータを、この実験の入力データや実験ソースコードとして用意します。  

～ データガバナンス機能のデータ管理方法について ～  

データガバナンス機能では、大容量データの管理にあたって、ファイルのリンク情報と実データを分けて管理する技術「git-annex」を採用しています。  
それにより、軽容量なリンク情報をGIN-forkのリポジトリに保存することで、実験実行環境から実データにアクセス・ダウンロードすることができます。
  
以下は操作の概要図です。   

<img src="./../../../files/WORKFLOWS/EX-WORKFLOWS/images/S3ユースケース.png" width=80%>


<!-- ## 1. S3ストレージから入力データや実験ソースコードを用意する -->
必要な処理を以下から選択し、実行してください。
- A：S3にある単一データを用意する場合
- B：複数のデータを取得する場合

## A：S3にある単一データを用意する場合

### A-1. S3にあるデータのオブジェクトURLと、そのデータの格納先を入力する
以下のセルを実行して、S3にあるデータのオブジェクトURLと、そのデータの格納先についての実験パッケージからファイル名までのパスを入力してください。  
※入力に誤りがある場合は、再度このセルを実行して下さい。  
<以下は例です。>
- オブジェクトURL：https://[bucket-title]].s3.ap-northeast-1.amazonaws.com/sample.txt  
- 格納先のファイルパス：input_data/s3/sample.txt  
<p style="color:red;">格納先のファイルパスは、`input_data/`もしくは`source/`で始まる必要があります。</p> 

In [ ]:
from ipywidgets import Text, Button, Layout
style = {'description_width': 'initial'}
%store -r EXPERIMENT_TITLE

def on_click_callback(clicked_button: Button) -> None:
    global input_url
    global input_path
    input_url = text_url.value
    input_path = text_path.value
    if input_path.startswith('input_data/') or input_path.startswith('source/'):
        button.description='入力を完了しました。'
        button.layout=Layout(width='250px')
        button.button_style='success'
    else:
        button.description='`input_data/`か`source/`で始まる必要があります。修正後、再度クリックしてください。'
        button.layout=Layout(width='700px')
        button.button_style='danger'

# テキストボックス
text_path = Text(
    description='*格納先のファイルパス：',
    placeholder='input_data/S3/sample.txt',
    layout=Layout(width='700px'),
    style=style
)
text_url = Text(
    description='*S3にあるデータのオブジェクトURL：',
    placeholder='https://[bucket-title]].s3.ap-northeast-1.amazonaws.com/sample.txt',
    layout=Layout(width='700px'),
    style=style
)
button = Button(description='入力を完了する',layout=Layout(width='250px'))
button.on_click(on_click_callback)
text_url.on_submit(on_click_callback)
display(text_url, text_path, button)

### A-2. リンクを作成する
上のセルで入力いただいた内容から、git-annex管理用のリンクを作成します。

In [ ]:
import traceback
from colorama import Fore

dest_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE + '/' + input_path
try:
    result = !git annex addurl --fast --file=$dest_path $input_url
    # 処理に失敗しても例外が発生しないため、以下の処理で失敗時に例外を発生させる
    for line in result:
        if 'failed' in line:
            raise Exception
except Exception:
    print(Fore.RED + 'リンク情報の作成に失敗しました。入力値を確認してください。\n')
    print(traceback.format_exc())
else:
    print(Fore.BLACK + 'リンク情報の作成に成功しました。\n')
finally:
    for line in result:
        print(line)

### A-3. データの来歴の記録と、実データのダウンロードを行う

In [ ]:
import os
import traceback
from datalad import api
from IPython.display import HTML, display, clear_output

git_path = []
try:
    # 来歴を残すために一旦annex管理でコミットする
    os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
    from util.scripts import utils
    os.chdir(os.environ['HOME'])
    utils.syncs_with_repo(git_path=None, gitannex_path=dest_path, message='S3ストレージから実験のデータを用意')
    
    # 実データを取得する
    api.get(path=dest_path)

    # sourceであれば、git管理にしてgitのコミット対象リストに加える
    if input_path.startswith('source/'):
        !git annex unannex $dest_path
        git_path.append(dest_path)
except Exception:
    display(HTML("<p><font color='red'>処理に失敗しました。用意したいデータにアクセス可能か確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    clear_output()
    display(HTML("<p>来歴の記録とデータのダウンロードに成功しました。次の処理にお進みください。</p>"))


### A-4. GIN-forkのリポジトリに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

git_path.append('WORKFLOWS/EX-WORKFLOWS/prepare_from_s3.ipynb')

utils.syncs_with_repo(git_path=git_path, gitannex_path=None, message=EXPERIMENT_TITLE + '_実験データの用意')

### A-5. 実験ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。 

## B：複数のデータを取得する場合

S3のバケットまたはバケット内のフォルダ単位でデータを取得します。  

### B-1. AWSと接続するための情報を入力する

フォルダ単位でデータを取得したい場合は、`任意のフォルダパス`にフォルダパスを入力してください。

In [ ]:
from ipywidgets import Text, Button, Layout, Password
import boto3

style = {'description_width': 'initial'}

def on_click_callback(clicked_button: Button) -> None:
    button.description='入力を受け付けました。'
    button.button_style='success'

# テキストボックス
input_aws_access_key_id = Password(
    description='*AWS_ACCESS_KEY_ID：',
    layout=Layout(width='700px'),
    style=style
)
input_aws_secret_access_key = Password(
    description='*AWS_SECRET_ACCESS_KEY：',
    layout=Layout(width='700px'),
    style=style
)
input_aws_default_region = Text(
    description='*AWS_DEFAULT_REGION：',
    placeholder='ap-northeast-1',
    layout=Layout(width='700px'),
    style=style
)
input_bucket_name = Text(
    description='*バケット名：',
    layout=Layout(width='700px'),
    style=style
)
input_prefix = Text(
    description='バケットの任意のフォルダパス：',
    placeholder='sample_a/sample_a_1',
    layout=Layout(width='700px'),
    style=style
)
button = Button(description='入力を完了する', layout=Layout(width='200px'))
button.on_click(on_click_callback)
display(input_aws_access_key_id, input_aws_secret_access_key, input_aws_default_region, input_bucket_name, input_prefix, button)

### B-2. 入力されたS3バケットに接続する

In [ ]:
import traceback
from IPython.display import HTML, display
import boto3

aws_default_region = input_aws_default_region.value
bucket_name = input_bucket_name.value
prefix = input_prefix.value
paths=[]

try:
    s3 = boto3.resource(
        's3',
        aws_access_key_id = input_aws_access_key_id.value,
        aws_secret_access_key = input_aws_secret_access_key.value,
    ) 
    bucket = s3.Bucket(bucket_name)
    if len(prefix)==0:
        response = bucket.meta.client.list_objects_v2(Bucket=bucket.name)
    else:
        response = bucket.meta.client.list_objects_v2(Bucket=bucket.name, Prefix=prefix)
    for content in response['Contents']:
        paths.append(content['Key'])
except Exception:
    display(HTML("<p><font color='red'>S3バケットに接続できません。手順B-1の入力に間違いがないか確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    display(HTML("<p>入力されたS3バケットに接続できました。次の処理にお進みください。</p>"))

### B-3. 必要なデータを選択する

以下のセルを実行すると、手順B-1で入力したS3バケットにあるデータが表示されます。  
表示されたデータから、この実験に必要な入力データやソースコードをクリックして選択してください。  
Ctrlキーを押しながらクリックすると複数のデータを選択できます。

In [ ]:
import panel as pn
pn.extension()
column = pn.Column()
gui_list = []

## フォルダをpathsから排除したあと、GUIを作成する
for path in paths:
    if path.endswith('/'):
        pass
    else:
        gui_list.append(path)
        
def generate_dest_list(event):
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    global dest_list
    dest_list = []
    for i in range(len(column)):
        if len(column[i].value) > 0:
            dest_list.append('### ' + column[i].name)
        for index in range(len(column[i].value)):
            dest_list.append(pn.widgets.TextInput(name=column[i].value[index], placeholder='Enter a file path here...'))
    
column.append(pn.widgets.MultiSelect(name = "S3ファイル", options=gui_list, size=len(gui_list)))
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(generate_dest_list)
column.append(done_button)
column

### B-4. 選択したデータの格納先を入力する

手順B-3で選択したデータについて、格納先を実験パッケージからファイル名までのパスで入力してください。  
例えば、`input_data/experiment_1_output/sample.txt`などです。  
<p style="color:red;">格納先のファイルパスは、`input_data/`, `source/`で始まる必要があります。</p> 

In [ ]:
import panel as pn
from IPython.display import HTML, display

# 選択完了ボタン押下後
def verify_input_text(event):
    for i in range(len(column)):
        panel_type = str(type(column[i]))
        if 'TextInput' in panel_type:
            if column[i].value.startswith('input_data/') or column[i].value.startswith('source/'):
                done_button.button_type = "success"
                done_button.name = "入力を完了しました。次の処理にお進みください。"
            else:
                done_button.button_type = "danger"
                done_button.name = "`input_data/`か`source/`で始まる必要があります。修正後、再度クリックしてください。"
                break
            
done_button = pn.widgets.Button(name= "入力を完了する", button_type= "primary")
done_button.on_click(verify_input_text)
column = pn.Column()
for gui in dest_list:
    column.append(gui)
column.append(done_button)
column

### B-5. 選択したデータのリンクを指定した格納先に用意する

In [ ]:
import os
import csv
import traceback
from datalad import api
from IPython.display import HTML, display

# datalad addurlsで使うファイルのダウンロードリンクと格納先のcsvファイルを生成する
objects=[]
%store -r EXPERIMENT_TITLE
package_path = 'experiments/' + EXPERIMENT_TITLE

for i in range(len(column)):
    panel_type = str(type(column[i]))
    if 'TextInput' in panel_type:
        url = "https://%s.s3.%s.amazonaws.com/%s" % (
            bucket_name,
            aws_default_region,
            column[i].name
        )
        objects.append([package_path + '/' + column[i].value, url])
        
os.chdir(os.environ['HOME'])
!mkdir -p .tmp
with open('/home/jovyan/.tmp/datalad-addurls.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['who','link'])
    for obj in objects:
        writer.writerow([obj[0],obj[1]])

# addurlsを実行してgit annexリンクを作成する
result = ''
try:
    result = !datalad addurls --nosave --fast .tmp/datalad-addurls.csv '{link}' '{who}'
    for line in result:
        if 'addurls(error)' in line  or 'addurls(impossible)' in line:
            raise Exception
except Exception:
    display(HTML("<p><font color='red'>リンクの作成に失敗しました。用意したいデータにアクセス可能か確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    display(HTML("<p>リンクの作成に成功しました。次の処理にお進みください。</p>"))

### B-6. データの来歴の記録と、実データのダウンロードを行う¶

In [ ]:
from datalad import api
from IPython.display import HTML, display, clear_output
import traceback

try:
    # このタスクで用意したファイルパス
    datalad_get_path = []
    for obj in objects:
        datalad_get_path.append(obj[0])

    # sourceの来歴を残すため一旦annex管理でコミットする
    api.save(path=datalad_get_path, message='S3ストレージの' + bucket_name + 'バケットから実験のデータを用意')

    # 実データを取得する
    api.get(path=datalad_get_path)

    # sourceがあれば、git管理にする
    source_path = []
    for path in datalad_get_path:
        if path.startswith('experiments/' + EXPERIMENT_TITLE + '/source'):
            source_path.append(path)
    if len(source_path) > 0:
        for path in source_path:
            !git annex unannex $path
except Exception:
    display(HTML("<p><font color='red'>処理に失敗しました。用意したいデータにアクセス可能か確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    clear_output()
    display(HTML("<p>来歴の記録とデータのダウンロードに成功しました。次の処理にお進みください。</p>"))

### B-7. GIN-forkのリポジトリに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

git_path = source_path
git_path.append('WORKFLOWS/EX-WORKFLOWS/prepare_from_s3.ipynb')

utils.syncs_with_repo(git_path=git_path, gitannex_path=None, message=EXPERIMENT_TITLE + '_実験データの用意')

### B-8. 実験ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。